In [1]:
import warnings
import os
import sys
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

In [2]:
path = '/home/zhouzr/data/tianchi/user_location_predict'
os.chdir(path)

In [8]:
train_sample = pd.read_csv('./original_data/训练数据-ccf_first_round_user_shop_behavior.csv')
shop = pd.read_csv('./original_data/训练数据-ccf_first_round_shop_info.csv')
test_sample = pd.read_csv('./original_data/AB榜测试集-evaluation_public.csv')

In [13]:
shop.rename(columns={'longitude': 'lgt', 'latitude': 'ltt', 'time_stamp': 'time'}, inplace=True)
train_sample.rename(columns={'longitude': 'lgt', 'latitude': 'ltt', 'time_stamp': 'time'}, inplace=True)
test_sample.rename(columns={'longitude': 'lgt', 'latitude': 'ltt', 'time_stamp': 'time'}, inplace=True)
train_sample = train_sample.merge(shop[['shop_id','mall_id']], on='shop_id', how='left')

In [16]:
sample = pd.concat([train_sample, test_sample])
sample['sample_id'] = range(sample.shape[0])
sample = sample.loc[:, ['sample_id','row_id','mall_id','user_id','lgt','ltt','time','wifi_infos','shop_id']]
sample['time'] = pd.to_datetime(sample['time'])

In [26]:
sample.to_csv('./preprocessing_data/sample.csv', index=None)
shop.to_csv('./preprocessing_data/shop.csv', index=None)

In [3]:
sample = pd.read_csv('./preprocessing_data/sample.csv')

/home/zhouzr/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
sample['time'] = pd.to_datetime(sample['time'])

In [9]:
sample_m1409 = sample[sample.mall_id=='m_1409'].copy().reset_index(drop=True)

In [10]:
def extract_space_feat(df):
    scaler = MinMaxScaler()
    feat = scaler.fit_transform(df[['lgt','ltt']])
    return feat

def extract_time_feat(df):
    df['weekday'] = df.time.dt.weekday
    df['hour'] = df.time.dt.hour
    scaler = MinMaxScaler()
    feat = scaler.fit_transform(df[['weekday','hour']])
    return feat

def extract_wifi_feat(df):
    scaler = MinMaxScaler()
    sample_id_list = []
    wifi_id_list = []
    signal_power_list = []
    signal_flag_list = []

    df = df.reset_index(drop=True)
    sample_size = df.shape[0]

    for i in range(sample_size):
        wifi_info = df.wifi_infos[i].split(';')
        sample_id = df.sample_id[i]
        for w in wifi_info:
            w_values = w.split('|')
            wifi_id_list.append(w_values[0])
            signal_power_list.append(float(w_values[1]))
            signal_flag_list.append(w_values[2])
            sample_id_list.append(sample_id)

    feat = pd.DataFrame({'sample_id': sample_id_list,
                         'signal_power': signal_power_list,
                         'signal_flag': signal_flag_list,
                         'wifi_id': wifi_id_list},
                        columns=['sample_id', 'wifi_id', 'signal_power', 'signal_flag'])
    min_power = feat.signal_power.min() - 1
    max_power = feat.signal_power.max()

    feat['signal_flag'] = feat.signal_flag.apply(lambda x: 1 if x == 'true' else 0)
    feat['signal_power'] = (feat['signal_power'] - min_power) / max_power

    wifi_power = pd.pivot_table(feat, index='sample_id', columns='wifi_id', values='signal_power')
    wifi_flag = pd.pivot_table(feat, index='sample_id', columns='wifi_id', values='signal_flag')
    wifi_feat = pd.concat([wifi_power, wifi_flag], axis=1).reset_index().fillna(0).drop('sample_id', axis=1).values
    wifi_feat = scaler.fit_transform(wifi_feat)
    return wifi_feat

In [31]:
import time
t1 = time.time()
f2 = extract_space_feat(sample_m1409)
f1 = extract_time_feat(sample_m1409)
f3 = extract_wifi_feat(sample_m1409)
t2 = time.time()
feat = np.concatenate([f1,f2,f3], axis=1)

train_idx = sample_m1409[sample_m1409.row_id.isnull()].index.tolist()

train_x = feat[train_idx]
train_y = sample_m1409[sample_m1409.row_id.isnull()]['shop_id'].values

model = LogisticRegression()
model.fit(train_x, train_y)
print('extract feature cost time %.2f'%(t2-t1))
print('train model cost time %.2f'%(time.time()-t2))

extract feature cost time 8.36
train model cost time 1421.71


In [32]:
model.score(train_x, train_y)

0.96890726096333568